# Import libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.core.debugger import set_trace
import json
from matplotlib import colors

# Load data

In [4]:
PATH = "../data/asocserreprivka-data-psynet/data/"
nodes = pd.read_csv(PATH + "node.csv", low_memory=False)
networks = pd.read_csv(PATH + "network.csv", low_memory=False)

# Preprocess data

In [5]:
# filter networks
network_data = networks
network_data = network_data[network_data["role"] == "experiment"]
network_data = network_data[network_data["failed"] == 'f']
network_data = network_data[network_data["trial_maker_id"] == 'graph_experiment']

experiment_net_id = list(network_data['id'].to_numpy())

# filter nodes
node_data = nodes
node_data = node_data[nodes["type"] == "graph_chain_node"]
node_data = node_data[node_data["failed"] == "f"]
node_data = node_data[node_data["network_id"].isin(experiment_net_id)]
node_data = node_data[["id", "network_id", "degree", "definition", "seed", "vertex_id", "dependent_vertex_ids"]]
node_data = node_data.sort_values(["network_id", "degree"])

In [6]:
node_data.to_csv("rivka-asocial-necklace-rep-data.csv")

In [7]:
# MOST COMMON NECKLACES ANALYSIS

def clean_json(x):
    return json.loads(x) # turns json into python object

In [8]:
# Most common necklaces that people produced
node_data = node_data.sort_values(["degree"])

In [9]:
responses = pd.read_csv(PATH + "response.csv", low_memory=False)

In [10]:
response_data = responses[responses["failed"] == "f"]
response_data = response_data[responses["question"] == "reproduce"]
answers = response_data["answer"]

In [17]:
def convert_to_pattern(necklace):
    color_to_code = {}
    pattern = []
    color_codes = ["a", "b", "c", "ERR"]
    color_index = 0

    if isinstance(necklace, str):
        necklace = necklace.strip('][').split(', ')

    for circle_color in necklace:
        if circle_color not in color_to_code:
            color_to_code[circle_color] = color_codes[color_index]
            color_index += 1
        pattern.append(color_to_code[circle_color])
    
    return str(pattern)

In [18]:
necklace_freqs = {}
for necklace in answers:
    necklace = convert_to_pattern(necklace)
    if necklace in necklace_freqs:
        necklace_freqs[necklace] += 1
    else:
        necklace_freqs[necklace] = 1

In [19]:
sorted_necklaces = sorted( ((v, k) for k, v in necklace_freqs.items()), reverse=True)

In [20]:
pd.DataFrame(sorted_necklaces).to_csv("asocial-necklace-patterns-by-frequency.csv")